In [1]:
from bs4 import BeautifulSoup
from zipfile import ZipFile
import os

In [28]:
datadir = "data"


def open_zip(datadir):
    with ZipFile('{0}.zip'.format(datadir), 'r') as myzip:
        myzip.extractall()


def process_all(datadir):
    files = os.listdir(datadir)
    return files



def test():
    print "Running a simple test..."
#    open_zip(datadir)
    files = process_all(datadir)
    data = []
    for f in files:
        data += process_file(f)
    assert len(data) == 3
    for entry in data[:3]:
        assert type(entry["year"]) == int
        assert type(entry["flights"]["domestic"]) == int
        assert len(entry["airport"]) == 3
        assert len(entry["courier"]) == 2
    print "... success!"


In [45]:
#open_zip(datadir)
import collections
airline_data = collections.namedtuple('airline_data','year month domestic international total')
files = process_all(datadir)
data = []
for f in files:
    data = []
    info = {}
    info["courier"], info["airport"] = f[:6].split("-")
    with open("{}/{}".format(datadir, f), "r") as html:
        soup = BeautifulSoup(html)
    for item in soup.select("tr.dataTDRight"):
        row_tuple = airline_data(*[el.string.replace(',','') for el in item.children if u'\n' not in el.string])
        if row_tuple.month == 'TOTAL':
            continue
        dict_info = { "courier": info["courier"],
                      "airport": info["airport"],
                      "year": int(row_tuple.year),
                      "month": int(row_tuple.month),
                      "flights": {"domestic": int(row_tuple.domestic),
                                  "international": int(row_tuple.international)}}
        data.append(dict_info)
    print data

[{'airport': 'ATL', 'month': 10, 'flights': {'international': 92565, 'domestic': 815489}, 'courier': 'FL', 'year': 2002}, {'airport': 'ATL', 'month': 11, 'flights': {'international': 91342, 'domestic': 766775}, 'courier': 'FL', 'year': 2002}, {'airport': 'ATL', 'month': 12, 'flights': {'international': 96881, 'domestic': 782175}, 'courier': 'FL', 'year': 2002}]


In [47]:
def process_file(f):
    # This is example of the datastructure you should return
    # Each item in the list should be a dictionary containing all the relevant data
    # Note - year, month, and the flight data should be integers
    # You should skip the rows that contain the TOTAL data for a year
    # data = [{"courier": "FL",
    #         "airport": "ATL",
    #         "year": 2012,
    #         "month": 12,
    #         "flights": {"domestic": 100,
    #                     "international": 100}
    #         },
    #         {"courier": "..."}
    # ]
    data = []
    info = {}
    info["courier"], info["airport"] = f[:6].split("-")
    
    with open("{}/{}".format(datadir, f), "r") as html:
        soup = BeautifulSoup(html)

    for item in soup.select("tr.dataTDRight"):
        row_tuple = airline_data(*[el.string.replace(',','') for el in item.children 
                                   if u'\n' not in el.string])
        if row_tuple.month == 'TOTAL':
            continue
        dict_info = { "courier": info["courier"],
                      "airport": info["airport"],
                      "year": int(row_tuple.year),
                      "month": int(row_tuple.month),
                      "flights": {"domestic": int(row_tuple.domestic),
                                  "international": int(row_tuple.international)}}
        data.append(dict_info)
        
    return data

In [ ]:
test()